## Purpose of this notebook is to demonstrate the full survey shot point processing, key point identification and plotting.

1. Import Text File of Survey Shot Data Points
2. Process Survey Shots to include Geometric relationships from centerline_feature_creator Module
3. Run pretrained shot categorization model
4. Plot

In [28]:
pd.options.display.max_rows = 10
import sys
import pandas as pd
import numpy as np
import json
from pprint import pprint
# sys.path

In [5]:
import_path = '/Users/connortluck/Desktop/Roadway_Geometry_Designed_Fit/Import/Horizontal_Accuracy_Testing_Data/horizontal_test_geometry_clean.txt'
df = pd.read_csv(import_path, sep=",", header='infer')
df.columns = ["Northing", "Easting", "Elevation"]
df['Elevation'] = 0

In [6]:
df.head(2)

,Northing,Easting,Elevation
0,315687.6432,60998.6484,0
1,315679.5908,60998.6323,0


In [7]:
'''create features from just northing easting and elevation import'''
import Modules.centerline_feature_creator
r = Modules.centerline_feature_creator.centerline_feature_creator(df)
df = r.output

/opt/anaconda3/lib/python3.7/site-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [9]:
df.head(2)

,Northing,Easting,Latitude,Longitude,Elevation,bearing_angles,three_point_sum,five_point_sum,segment_length,circonscrit_circle_radius,oscillating_circle_radius
0,315687.6432,60998.6484,40.308985,-76.397378,0,0.000000,0.000000,0.0,8.052416,0.000000e+00,0.000000e+00
1,315679.5908,60998.6323,40.308984,-76.397407,0,0.000001,0.000002,0.0,8.052316,3.242983e+08,3.264859e+08


In [11]:
'''LOAD THE MODEL'''
import pickle
loaded_model = pickle.load(open('/Users/connortluck/Desktop/Roadway_Geometry_Designed_Fit/Modules/finalized_model.sav', 'rb'))

#create the fatures which match the premade model. 
x = df[["bearing_angles",'three_point_sum','five_point_sum','segment_length','circonscrit_circle_radius','oscillating_circle_radius']]

#predict the class
y_pred = loaded_model.predict(x)

#append the class to the dataframe
df['class'] = y_pred


In [12]:
'''filter outliers from model'''
df = Modules.centerline_feature_creator.class_outliers(df)

Point filtered: Index position: 137 Original point: 1, Changed to: 0.0
Point filtered: Index position: 245 Original point: 0, Changed to: 1.0


In [14]:
df.head(1)

,Northing,Easting,Latitude,Longitude,Elevation,bearing_angles,three_point_sum,five_point_sum,segment_length,circonscrit_circle_radius,oscillating_circle_radius,class
0,315687.6432,60998.6484,40.308985,-76.397378,0,0.0,0.0,0.0,8.052416,0.0,0.0,0


In [71]:
# working_clean_x = cleaned_points.cleaned_x
# clean_curve_y(working_clean_x,sc.list_of_curves,sc.best_fit_radius_list,sc.curve_geometry_all)

In [15]:
'''establish geometry from classified dataframe'''
import Modules.geometry_cleaner
sc = Modules.geometry_cleaner.Processed_Geometry(df)

In [16]:
'''CLEAN geometry from classified dataframe'''
cleaned = Modules.geometry_cleaner.cleaned(sc)

In [61]:
'''attributes for sc class object'''
{ k for k in dir(sc) if not k.startswith('_') }

{'Points_of_intersection_list',
 'best_fit_radius_list',
 'best_fit_tangent_list',
 'best_fit_tangents',
 'calc_R',
 'curve_center_finder',
 'curve_estimated_centers',
 'curve_geometry_all',
 'curve_geometry_combined',
 'dataframe',
 'f',
 'leastsq_circle',
 'list_of_PI',
 'list_of_curves',
 'list_of_tangents',
 'radius_fit',
 'tangent_sort'}

In [62]:
'''full report for sc class object'''
print(vars(sc))

{'dataframe':         Northing     Easting   Latitude  Longitude  Elevation  bearing_angles  \
0    315687.6432  60998.6484  40.308985 -76.397378          0    0.000000e+00   
1    315679.5908  60998.6323  40.308984 -76.397407          0    1.207418e-06   
2    315671.5385  60998.6162  40.308984 -76.397436          0    1.207418e-06   
3    315663.4861  60998.6001  40.308983 -76.397465          0    1.207418e-06   
4    315655.4338  60998.5840  40.308982 -76.397494          0    7.101108e-04   
..           ...         ...        ...        ...        ...             ...   
295  314262.7277  60802.5198  40.308340 -76.402466          0    6.685675e-04   
296  314255.7734  60799.1678  40.308330 -76.402491          0    8.537736e-07   
297  314248.8191  60795.8158  40.308320 -76.402515          0    6.685680e-04   
298  314241.8648  60792.4639  40.308311 -76.402540          0    6.685685e-04   
299  314234.9105  60789.1119  40.308301 -76.402564          0    1.000000e-14   

     three_po

In [17]:
# test_radii = sc.best_fit_radius_list

## Now must clean up the "dirty" survey points by using clean spacing along x axis and calculating the new y points.

In [19]:
'''dnd this code is THE code'''
def clean_curve_x(dirty_curves):
    clean_x_pts_curve=[]
    #use linspace to generate clean x points with first and last cooridates.
    for curves in dirty_curves:
        clean_x_curve = np.linspace(curves[0][0],curves[-1][0],30)
        clean_x_pts_curve.append(clean_x_curve)
    return(clean_x_pts_curve)

def clean_curve_y(clean_x_pts_curve):
    clean_y_pts_curve = []
    append_y_pts_curve = []
    for curve in range(len(clean_x_pts_curve)):
        r = sc.best_fit_radius_list[curve]
        xc = sc.curve_geometry_all[curve][1][0]
        yc = sc.curve_geometry_all[curve][1][1]
        
        for point in clean_x_pts_curve[curve]:
            if abs(point) - abs(xc) < 0 :
#                 y = yc + sqrt((r**2)-((point-xc)**2))
                y = yc + sqrt((r**2)-((point-xc)**2))
                append_y_pts_curve.append((point,y))
            else:
#                 y = yc - sqrt((r**2)-((point-xc)**2))
                y1 = yc - sqrt((r**2)-((point-xc)**2))
                if abs(y) - abs(yc) < 0:
                    y1 = yc + sqrt((r**2)-((point-xc)**2))
                    append_y_pts_curve.append((point,y1))
                else:
                    pass
                append_y_pts_curve.append((point,y1))
                
            clean_y_pts_curve.append(append_y_pts_curve)
            append_y_pts_curve=[]
    return(clean_y_pts_curve)

In [20]:
from math import sqrt
'''DND'''
cleaned_x = clean_curve_x(sc.list_of_curves)
cleaned_y = clean_curve_y(cleaned_x)


In [21]:
'''DND'''
cleaned_curves = []
for i in cleaned_y:
    for g in i:
        cleaned_curves.append(g)
cc_df = pd.DataFrame(data = cleaned_curves,columns = ["Easting","Northing"])
cc_df.head(400)

,Easting,Northing
0,60998.253700,315479.997037
1,61000.151190,315512.075473
2,61002.048679,315526.996300
3,61003.946169,315538.424856
4,61005.843659,315548.028652
...,...,...
175,60882.380172,315202.159065
176,60882.087886,315202.772325
177,60882.087886,315202.772325
178,60881.795600,315203.384396


In [22]:
cc_df.keys()

Index(['Easting', 'Northing'], dtype='object')

In [59]:
sc.curve_estimated_centers

[(61348.23643419413, 315475.3903492654),
 (60822.331840767634, 315091.09862782),
 (60828.65296797991, 314954.7548515135),
 (61294.01359058765, 314470.84773970424),
 (60069.89862845269, 314815.29794438457)]

In [87]:
center_of_curve_estimate_list = [(-1197419.1957692867, -634953.3406369429),
 (-1197843.660418561, -633364.5173241962),
 (-1198113.1413177005, -634150.5832759687),
 (-1198452.4996548505, -634483.3170877026)]

In [88]:
cce = pd.DataFrame(center_of_curve_estimate_list,columns = [["Easting","Northing"]])

In [89]:

import pyproj
#define coord systems. 
wgs84=pyproj.CRS("EPSG:4326") # LatLon with WGS84 datum used by GPS units and Google Earth 
krov=pyproj.CRS("EPSG:5514") # russia? 
#lat is northing. 
geo_lat = []
geo_long = []
for index,row in cce.iterrows():
    geo_lat.append(row['Northing'])
    geo_long.append(row['Easting'])

    
converted = pyproj.transform(krov, wgs84, geo_lat,geo_long)

cce['Latitude'] = converted[0]
cce['Longitude'] = converted[1]

cce.head(4)

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:13: DeprecationWarning: This function is deprecated. See: https://pyproj4.github.io/pyproj/stable/gotchas.html#upgrading-to-pyproj-2-from-pyproj-1
  del sys.path[0]


,Easting,Northing,Latitude,Longitude
0,-1.197419e+06,-634953.340637,48.830866,16.163435
1,-1.197844e+06,-633364.517324,48.828690,16.185592
2,-1.198113e+06,-634150.583276,48.825483,16.175369
3,-1.198452e+06,-634483.317088,48.822112,16.171390


In [90]:

import pyproj
#define coord systems. 
wgs84=pyproj.CRS("EPSG:4326") # LatLon with WGS84 datum used by GPS units and Google Earth 
krov=pyproj.CRS("EPSG:5514") # russia? 
#lat is northing. 
geo_lat = []
geo_long = []
for index,row in cc_df.iterrows():
    geo_lat.append(row['Northing'])
    geo_long.append(row['Easting'])

    
converted = pyproj.transform(krov, wgs84, geo_lat,geo_long)

cc_df['Latitude'] = converted[0]
cc_df['Longitude'] = converted[1]

cc_df.head(4)

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:13: DeprecationWarning: This function is deprecated. See: https://pyproj4.github.io/pyproj/stable/gotchas.html#upgrading-to-pyproj-2-from-pyproj-1
  del sys.path[0]


,Easting,Northing,Latitude,Longitude
0,60998.253700,315479.997037,60.269305,30.277174
1,61000.151190,315512.075473,60.269309,30.277744
2,61002.048679,315526.996300,60.269320,30.278010
3,61003.946169,315538.424856,60.269332,30.278214


In [91]:
pi_list = [(-1197366.2053365274, -634687.0616027372),
 (-1198134.8857416897, -634548.0483255866),
 (-1198258.7977614298, -634509.5147132258),
 (-1198428.1507028595, -634399.1408386733)]

In [92]:
pi_df = pd.DataFrame(pi_list,columns = ['Easting','Northing'])


import pyproj
#define coord systems. 
wgs84=pyproj.CRS("EPSG:4326") # LatLon with WGS84 datum used by GPS units and Google Earth 
krov=pyproj.CRS("EPSG:5514") # russia? 
#lat is northing. 
geo_lat = []
geo_long = []
for index,row in pi_df.iterrows():
    geo_lat.append(row['Northing'])
    geo_long.append(row['Easting'])

    
converted = pyproj.transform(krov, wgs84, geo_lat,geo_long)

pi_df['Latitude'] = converted[0]
pi_df['Longitude'] = converted[1]

pi_df.head(4)

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:16: DeprecationWarning: This function is deprecated. See: https://pyproj4.github.io/pyproj/stable/gotchas.html#upgrading-to-pyproj-2-from-pyproj-1
  app.launch_new_instance()


,Easting,Northing,Latitude,Longitude
0,-1.197366e+06,-634687.061603,48.831610,16.166957
1,-1.198135e+06,-634548.048326,48.824884,16.170024
2,-1.198259e+06,-634509.514713,48.823816,16.170737
3,-1.198428e+06,-634399.140839,48.822415,16.172491


In [93]:
'''sc and cleaned_points are the two objects'''

'sc and cleaned_points are the two objects'

In [98]:
cleaned.key_points[cleaned.key_points['Point Type'] == 'point_of_intersection']

,Easting,Northing,Point Type,Latitude,Longitude
0,60999.549744,315371.048002,point_of_intersection,40.308964,-76.398513
1,61118.169916,315184.426263,point_of_intersection,40.309275,-76.399194
2,61134.665328,314848.130165,point_of_intersection,40.309295,-76.400401
3,60968.872874,314663.441827,point_of_intersection,40.308826,-76.401046
4,60886.135505,314436.290806,point_of_intersection,40.308582,-76.401852


In [99]:
cleaned.key_points[cleaned.key_points['Point Type'] == 'point_of_intersection']

,Easting,Northing,Point Type,Latitude,Longitude
0,60999.549744,315371.048002,point_of_intersection,40.308964,-76.398513
1,61118.169916,315184.426263,point_of_intersection,40.309275,-76.399194
2,61134.665328,314848.130165,point_of_intersection,40.309295,-76.400401
3,60968.872874,314663.441827,point_of_intersection,40.308826,-76.401046
4,60886.135505,314436.290806,point_of_intersection,40.308582,-76.401852


In [101]:
tangent_test = cleaned.tangent_dataframe

In [102]:
"""All points graphed here, no training"""
import folium
folium_map_og = folium.Map(location=(40.308968, -76.399801),
                        zoom_start=20,
                        max_zoom = 40,
                        tiles='cartodbpositron')

    
    
# #map curves in red
# for index,row in df.iterrows():
#     color="#b30000"
#     if row['class'] == 1:
#         folium.CircleMarker(location=(row['Latitude'],row['Longitude']),
#                             radius=4,
#                             tooltip="the point type is: " + str(row['class']) + " and the curve id is: " + str(row['id']), 
#                             fill=True,
#                             color=color).add_to(folium_map_og) 
# #map tangents in blue
# for index,row in df.iterrows():
#     color="#0099ff"
#     if row['class'] == 0:

#         folium.CircleMarker(location=(row['Latitude'],row['Longitude']),
#                             radius=4,
#                             tooltip="the point type is: " + str(row['class']) + " and the curve id is: " + str(row['id']),
#                             fill=True,
#                             color=color).add_to(folium_map_og)  
        
# map PI in green
for index,row in cleaned.key_points[cleaned.key_points['Point Type'] == 'estimated_curve_center'].iterrows():
    color="#339237"
    folium.CircleMarker(location=(row['Latitude'],row['Longitude']),
                        radius=4,
                        fill=True,
                        color=color).add_to(folium_map_og)  
#map fitted in orange
for index,row in cleaned.key_points[cleaned.key_points['Point Type'] == 'point_of_intersection'].iterrows():
    color="#33f5ee"
    folium.CircleMarker(location=(row['Latitude'],row['Longitude']),
                        radius=4,
                        fill=True,
                        color=color,
                        popup=(row['Latitude'],row['Longitude'])).add_to(folium_map_og) 
    
# #map fitted in pink
# for index,row in cleaned_tangent_df.iterrows():
#     color="#FF69B4"
#     folium.CircleMarker(location=(row['Latitude'],row['Longitude']),
#                         radius=4,
#                         fill=True,
#                         color=color).add_to(folium_map_og) 
   
#clean_Curves
for index,row in cleaned.output.iterrows():
    color="#0000ff"
    folium.CircleMarker(location=(row['Latitude'],row['Longitude']),
                        radius=4,
                        fill=True,
                        color=color,
                        popup=(row['Latitude'],row['Longitude'])).add_to(folium_map_og) 
    
for index,row in tangent_test.iterrows():
    color= '#F39C12'
    folium.CircleMarker(location=(row['Latitude'],row['Longitude']),
                        radius=4,
                        fill=True,
                        color=color,
                        popup=(row['Latitude'],row['Longitude'])).add_to(folium_map_og) 


folium_map_og

    

# Create a Json File to output on the microservice server
1. Geometry and Raw Points

   1. Original DataFrame of Raw Points = sc.dataframe
   1. list of curves = sc.list_of_curves
   1. list of tangents = sc.list_of_tangents
   1. best fit radius list = sc.best_fit_radius_list
   1. best fit tangent list = sc.best_fit_tangent_list
   1. points of intersection list = sc.Point_of_intersection_list
   1. curve estimated centers = sc.curve_estimated_centers
   
1. Cleaned Geometry Point Outputs

   1. Item 3a
   1. Item 3b
   1. Original DataFrame of Raw Points = sc.dataframe
   1. list of curve
   1. Original DataFrame of Raw Points = sc.dataframe
   1. list of curve
   1. Original DataFrame of Raw Points = sc.dataframe
   1. list of curve


In [ ]:
cleaned.output

<p>&nbsp;</p>
<p>&nbsp;</p>

Name of Data | Class Callout
------------ | -------------
Original DataFrame of Raw Points | sc.dataframe
list of curves  | sc.list_of_curves
list of tangents | sc.list_of_tangents 
best fit radius list | sc.best_fit_radius_list
best fit tangent list | sc.best_fit_tangent_list
points of intersection list | sc.Point_of_intersection_list
curve estimated centers | sc.curve_estimated_centers



<p>&nbsp;</p>
<p>&nbsp;</p>


Name of Data | Class Callout
------------ | -------------
cleaned x | cleaned.cleaned_x
cleaned y | cleaned.cleaned_y
cleaned output | cleaned.output
cleaned tangent | cleaned.tangent_dataframe

<p>&nbsp;</p>
<p>&nbsp;</p>


In [103]:
# cleaned_output = {'clean x coordinates': cleaned.cleaned_x,
#                  'clean y coordinates': cleaned.cleaned_y,
#                  'output_all': cleaned.output,
#                  'cleaned tangent': cleaned.tangent_dataframe}

In [104]:
# output_geometry_data = {'Original DataFrame of Raw Points': sc.dataframe,
#                         'list of curves': sc.list_of_curves,
#                         'list of tangents': sc.list_of_tangents,
#                         'best fit radius list': sc.best_fit_radius_list,
#                         'best fit tangent list': sc.best_fit_tangent_list,
#                         'points of intersection list': sc.Points_of_intersection_list,
#                         'curve estiamted centers': sc.curve_estimated_centers}

In [109]:
print(output_geometry_data.keys())

dict_keys(['Original DataFrame of Raw Points', 'list of curves', 'list of tangents', 'best fit radius list', 'best fit tangent list', 'points of intersection list', 'curve estiamted centers'])


In [ ]:
cleaned_output = {'clean x coordinates': cleaned.cleaned_x,
                 'clean y coordinates': cleaned.cleaned_y,
                 'output_all': cleaned.output,
                 'cleaned tangent': cleaned.tangent_dataframe}

In [ ]:
output_geometry_data = {'Original DataFrame of Raw Points': sc.dataframe,
                        'list of curves': sc.list_of_curves,
                        'list of tangents': sc.list_of_tangents,
                        'best fit radius list': sc.best_fit_radius_list,
                        'best fit tangent list': sc.best_fit_tangent_list,
                        'points of intersection list': sc.Points_of_intersection_list,
                        'curve estiamted centers': sc.curve_estimated_centers}

In [159]:
# converted = []
# def ndarry_json(working_list):
#     for i in working_list:
#         converted.append(i.tolist())
#     return(converted)

In [170]:
# cleaned.cleaned_y

In [165]:
json.dumps(json.loads(cleaned.cleaned_y.to_json(orient='index')), indent=2)


AttributeError: 'list' object has no attribute 'to_json'